<a href="https://colab.research.google.com/github/yoonputer/Team_Project2/blob/master/Deeplearning/test_dummy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel('./drive/MyDrive/Forkspoon/test.xls')

In [ ]:
df

,company,group,clasificar
0,(주)포스코인터내셔널\n\n관심기업\n 2021년 상반기 신입 수출입·무역사무합격자...,전문가 자소서 총평\n\n\n\n\n\n 2\n ...,질문Q1.성장과정 및 생활신조 등 자기소개를 기술하여 주시기 바랍니다. 보기\n\n...


In [ ]:
g_data= df['group']
g_data = pd.DataFrame(g_data)

In [ ]:
c_data= df['clasificar']
c_data = pd.DataFrame(c_data)

In [ ]:
total_data= g_data.join(c_data)

In [ ]:
total_data

,group,clasificar
0,전문가 자소서 총평\n\n\n\n\n\n 2\n ...,질문Q1.성장과정 및 생활신조 등 자기소개를 기술하여 주시기 바랍니다. 보기\n\n...


In [ ]:
total_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   group       1 non-null      object
 1   clasificar  1 non-null      object
dtypes: object(2)
memory usage: 144.0+ bytes


In [ ]:
!python -m pip install konlpy

In [ ]:
!curl -O https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh

In [ ]:
!bash ./mecab.sh

In [ ]:
x_data = total_data['clasificar']
y_data = total_data['group']
x_data.shape , y_data.shape

((1,), (1,))

In [ ]:
from konlpy.tag import Mecab

In [ ]:
mecab =Mecab()

In [ ]:
print(mecab.morphs('성장과정 및 생활신조 등 자기소개를 기술하여 주시기 바랍니다.'))

['성장', '과정', '및', '생활신조', '등', '자기소개', '를', '기술', '하', '여', '주', '시', '기', '바랍니다', '.']
